In [22]:
# imports

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# import tensorflow as tf

# Splitting Up Data

In [23]:
# images only

labelled_image = pd.read_csv('labelled_images.csv')
labelled_image['Target'] = labelled_image['Target'].map({'Alex': 0, 'Kelly': 1})

X = labelled_image.drop(columns='Target')
y = labelled_image['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# full data with features

# images_df = pd.read_csv('images.csv')
# images_df['Target'] = images_df['Target'].map({'Alex': 0, 'Kelly': 1})

# X = images_df.drop(columns='Target')
# y = images_df['Target']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Convolutional Neural Network on Images Only

In [25]:
# reduce image size?

# import tensorflow as tf


# def load_images_from_paths(paths, img_size=(128, 128)):
#     images = []
#     for path in paths:
#         image = tf.io.read_file(path)
#         image = tf.image.decode_png(image, channels=3)
#         image = tf.image.resize(image, img_size)
#         image = image / 255.0  # Normalize
#         images.append(image.numpy())
#     return tf.convert_to_tensor(images)

# # Preprocess the train and test sets
# X_train_images = load_images_from_paths(X_train['Path'])
# X_test_images = load_images_from_paths(X_test['Path'])


In [26]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # TRAIN MODEL

# # Define CNN model
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.2),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(1, activation='sigmoid')  # Binary classification
# ])

# # compile model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # train model
# history = model.fit(
#     X_train_images,
#     y_train,
#     validation_data=(X_test_images, y_test),
#     epochs=10,
#     batch_size=32,
#     verbose=1
# )

In [27]:
# OR USE DATASET API

# def load_data_from_csv(csv_path):
#     df = pd.read_csv(csv_path)
#     df['Target'] = df['Target'].map({'Alex': 0, 'Kelly': 1})
#     return tf.data.Dataset.from_tensor_slices((df['Path'].values, df['Target'].values))

# def preprocess_image(image_path, label, img_size=(128, 128)):
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_png(image, channels=3)
#     image = tf.image.resize(image, img_size)
#     image = image / 255.0
#     return image, label

# train_dataset = load_data_from_csv('labelled_images.csv').map(preprocess_image).batch(32)